In [1]:
import math
import numpy as np

import torch
from torch import nn

In [2]:
# 最初はconfig辞書の作成

config = {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "tokenizer_class": "BertJapaneseTokenizer",
  "type_vocab_size": 2,
  "vocab_size": 32000
}
config['hidden_size']


768

In [3]:
# Embeddings
class BertEmbeddings(nn.Module):
    """
    Bertは三つの埋め込みがある、単語のため、トーケンタイプのため、位置情報のため
    単語IDに分散表現に入れて、その後トーケンタイプを足し算して、最後に位置情報も足し算します
    """
    def __init__(self, config):
        super().__init__()
        
        # Token Embedding: 単語IDをベクトル化、ボキャブラリーサイズは32000と隠れ次元は768
        self.word_embeddings = nn.Embedding(config["vocab_size"], config["hidden_size"],
                                           padding_idx=config["pad_token_id"])

        # Token Type Embedding:
        self.token_type_embeddings = nn.Embedding(config["type_vocab_size"], config["hidden_size"])
        
        # Position Embedding: 位置情報ベクトルを作る、このmax_position_embeddingsは文章の最大の長さ。
        self.position_embeddings = nn.Embedding(config["max_position_embeddings"], config["hidden_size"])

        self.LayerNorm = nn.LayerNorm(config["hidden_size"], eps=config["layer_norm_eps"])
        self.dropout = nn.Dropout(config["hidden_dropout_prob"])

    def forward(self, input_ids, token_type_ids=None, position_ids=None, inputs_embeds=None):
        """
        [batch_size, seq_length]
        input_ids: 単語IDの羅列です。
        token_type_ids: 一つ目の文章の場合なら、0、一つ目の文章の場合なら、1。
        position_ids: 位置情報。
        input_embeds: もし既に単語は分散表現の形で。[batch_size, hidden_size]
        """
        if input_ids is not None:
            # [batch_size, seq_length]
            input_shape = input_ids.size()
        else:
            input_shape = input_embeds.size()[:-1]
    
        seq_length = input_shape[1]
        
        # 1. Token Embedding
        words_embeddings = self.word_embeddings(input_ids)
    
        # 2. Token Type Embedding
        if token_type_ids is None:
            token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=input_ids.device)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)
    
        # 3. Position Embedding
        if position_ids is None:
            position_ids = torch.arange(
                    seq_length, dtype=torch.long, device=input_ids.device)
            # batch_sizeを入れて
            position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
    
        # 全部の埋め込みを足し算して[batch_size, seq_len, hidden_size]
        embeddings = words_embeddings + position_embeddings + token_type_embeddings
    
        # LayerNormとDropout
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings


In [4]:
a = torch.randint(3, 5, (1,256), dtype=torch.long)
embeddings = BertEmbeddings(config)

embeddings(a), embeddings(a).shape

(tensor([[[-0.8970,  1.8419, -0.4941,  ..., -0.8248,  3.2126,  0.2309],
          [-0.9316,  0.0000, -0.7703,  ..., -1.2291,  3.1939,  1.0356],
          [-1.1881,  0.9460, -0.1656,  ..., -1.3448,  2.9268,  0.3051],
          ...,
          [-0.4960,  2.3689, -0.0327,  ..., -1.1537,  1.9008, -1.9447],
          [ 0.0000,  1.5613,  0.0346,  ...,  0.5873,  1.7842,  0.0000],
          [-0.0523,  0.1368,  0.1890,  ..., -0.0105,  2.4563, -0.0000]]],
        grad_fn=<MulBackward0>),
 torch.Size([1, 256, 768]))

In [5]:
# Multi Head Self Attention
class BertSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_attention_heads = config["num_attention_heads"]
        self.attention_head_size = int(config["hidden_size"]/config["num_attention_heads"])
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        # Attentionの全結合層
        self.query = nn.Linear(config["hidden_size"], self.all_head_size)
        self.key = nn.Linear(config["hidden_size"], self.all_head_size)
        self.value = nn.Linear(config["hidden_size"], self.all_head_size)

        self.dropout = nn.Dropout(config["attention_probs_dropout_prob"])

    def transpose_for_scores(self, x):
        """
        Multi Head Attentionのためにテンソルの形を変わる。
            num_attention_heads=12
            hidden_size=768
        [batch_size, seq_len, hidden] -> [batch_size, num_attention_heads, seq_len, hidden/num_attention_heads]
        最初のHeadは隠れ次元０から63、二つ目のHeadは64から127、．．．
        """
        # x.size()[:-1]これは隠れ次元を除くしてる
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        
        return x.permute(0, 2, 1, 3)

    def forward(self, hidden_states, attention_mask=None, output_attention=False):
        """
        hidden_states: Embeddingsモジュールもしくは前段のBertLayerからの出力。[batch_size, seq_len, hidden]
        attention_mask: Paddingsはソフトマックスに影響を与えないようにする。[batch_size, 1, 1, seq_len]
        output_attention: Attentionの重みを出力するか
        """

        # 入力を全結合層で特徴量変換（注意、multi-head Attentionの全部をまとめて変換しています）
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(hidden_states)
        mixed_value_layer = self.value(hidden_states)

        # Headsを付ける[batch_size, seq_len, 768] -> [batch_size, 12, seq_len, 64]
        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        # Attentionを計算する、クエリーとキーを掛け算する、類似度を得る
        # [batch_size, 12, seq_len, 64] x [batch_size, 12, 64, seq_len] -> [batch_size, 12, seq_len, seq_len]
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / math.sqrt(self.attention_head_size)

        # マスクをかけます
        if attention_mask is not None:
            attention_scores = attention_scores + attention_mask
            # このマスクはBertModelのforwardで作成されます、この後ソフトマックスをかけるから
            # 無視したい値があるでしょう。そのために掛け算じゃない、足し算のはソフトマックスの
            # 仕組みのためです。SoftMax(0)は高い値かもしれない、けどSoftMax(-inf)なら必ず0になる。
            # attention_maskの形は[batch_size, 1, 1, seq_len]
            # それは全部の[num_heads, seq_len, seq_len]に足し算するpaddingの部分は興味ない。
            #だから全部のattention_scoresにマスクを使う
        
            #extended_attention_mask: torch.Tensor = self.get_extended_attention_mask(attention_mask, input_shape, device)
            # https://github.com/huggingface/transformers/blob/main/src/transformers/modeling_utils.py#L964

        # Attentionを正規化する、確率を得る
        attention_probs = nn.Softmax(dim=-1)(attention_scores)

        attention_probs = self.dropout(attention_probs)

        # Attention Mapを掛け算します
        context_layer = torch.matmul(attention_probs, value_layer)

        # 前の形に戻る[batch_size, 12, seq_len, 64]-> [batch_size, seq_len, 12, 64]
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        # [batch_size, seq_len, 12, 64] -> [batch_size, seq_len, 768]
        context_layer = context_layer.view(*new_context_layer_shape)

        outputs = (context_layer, attention_probs) if output_attention else (context_layer,)
        return outputs
        

In [6]:
query = torch.randn(1, 12, 128, 64)
key = torch.randn(1, 12, 128, 64)

torch.matmul(query, key.transpose(-1, -2)).shape, key.transpose(-1, -2).shape

(torch.Size([1, 12, 128, 128]), torch.Size([1, 12, 64, 128]))

MxN NxM -> MxM

In [7]:
class BertSelfoutput(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config["hidden_size"], config["hidden_size"])
        self.LayerNorm = nn.LayerNorm(config["hidden_size"], eps=config["layer_norm_eps"])
        self.dropout = nn.Dropout(config["hidden_dropout_prob"])

    def forward(self, hidden_states, input_tensor):
        # この全結合層はMulti-HeadAttentionの部分です
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        # Add and Normの部分
        hidden_states = self.LayerNorm(hidden_states + input_tensor)

        return hidden_states

In [8]:
# BertSelfAttentionとBertSelfOutputを使う
class BertAttention(nn.Module):
    '''BertLayerモジュールのSelf-AttentionとAdd and Normの部分です'''
    def __init__(self, config):
        super().__init__()
        self.self = BertSelfAttention(config)
        self.output = BertSelfoutput(config)

    def forward(self, hidden_states, attention_mask=None, output_attention=False):
        """
        hidden_states: Embeddingsモジュールもしくは前段のBertLayerからの出力。[batch_size, seq_len, hidden]
        attention_mask: Paddingsはソフトマックスに影響を与えないようにする。[batch_size, 1, 1, seq_len]
        output_attention: Attentionの重みを出力するか
        """
        self_outputs = self.self(hidden_states, attention_mask, output_attention)
        attention_output = self.output(self_outputs[0], hidden_states)

        # もしAttentionがある場合に追加する
        outputs = (attention_output,) + self_outputs[1:]
        return outputs

In [9]:
a = torch.randint(3, 5, (1,256), dtype=torch.long)
embeddings = BertEmbeddings(config)

input_tensor = embeddings(a)
attention = BertAttention(config)

attention(input_tensor), len(attention(input_tensor)), attention(input_tensor)[0].shape

((tensor([[[-0.2125, -1.0476, -0.6899,  ...,  0.6433,  1.2850, -2.1511],
           [-0.2007, -0.4193,  1.3287,  ..., -1.2475, -1.0911,  0.3013],
           [-0.4148, -0.6579, -0.0326,  ...,  0.0058,  2.0633, -0.1529],
           ...,
           [ 0.2762, -0.1312,  2.6012,  ..., -0.7131, -0.2553,  0.5749],
           [ 0.0839,  0.0355,  1.4692,  ...,  0.1210, -0.2201, -0.1695],
           [ 0.3934,  0.1386,  1.7520,  ...,  0.1448,  0.5831, -0.6635]]],
         grad_fn=<NativeLayerNormBackward0>),),
 1,
 torch.Size([1, 256, 768]))

In [10]:
outputs = attention(input_tensor, output_attention=True)
len(outputs), outputs[0].shape, outputs[1].shape

(2, torch.Size([1, 256, 768]), torch.Size([1, 12, 256, 256]))

In [11]:
# MLPの部分
class BertIntermediate(nn.Module):
    '''BertのTransformerBlockモジュールのFeedforwardです'''

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config["hidden_size"], config["intermediate_size"])
        self.intermediate_act_fn = nn.GELU()

    def forward(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states)
        return hidden_states
        

In [12]:
class BertOutput(nn.Module):
    """Add and Normの部分"""
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config["intermediate_size"], config["hidden_size"])
        self.LayerNorm = nn.LayerNorm(config["hidden_size"], eps=config["layer_norm_eps"])
        self.dropout = nn.Dropout(config["hidden_dropout_prob"])

    def forward(self, hidden_states, input_tensor):
        """
        hidden_states: BertIntermediateモジュールからの出力。[batch_size, seq_len, intermediate_size]
        input_tensor: Attentionモジュールからの出力。[batch_size, seq_len, hidden]
        """
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states

In [55]:
# 全部のモジュールを一つのクラスでまとめる。
class BertLayer(nn.Module):
    """Transformerブロックになります"""
    def __init__(self, config):
        super().__init__()
        
        # Self-Attention部分
        self.attention = BertAttention(config)

        # Self-Attentionの出力を処理する全結合層
        self.intermediate = BertIntermediate(config)

        # Self-Attentionよる特徴量とBertLayerへの元の入力を足し算する層
        self.output = BertOutput(config)

    def forward(self, hidden_states, attention_mask=None, output_attention=False):
        """
        hidden_states: Embeddingsモジュールもしくは前段のBertLayerからの出力。[batch_size, seq_len, hidden]
        attention_mask: Paddingsはソフトマックスに影響を与えないようにする。[batch_size, 1, 1, seq_len]
        output_attention: Attentionの重みを出力するか
        """

        self_attention_outputs = self.attention(
            hidden_states,
            attention_mask,
            output_attention=output_attention,
        )
        attention_output = self_attention_outputs[0]
        outputs = self_attention_outputs[1:]  # add self attentions if we output attention weights

        intermediate_output = self.intermediate(attention_output)
        layer_output = self.output(intermediate_output, attention_output)

        # [batch_size, seq_length, hidden_size]
        outputs = (layer_output,) + outputs
        return outputs


In [14]:
a = torch.zeros(1, 12, 128, 128)
mask = torch.ones(1, 1, 1, 128)
(a + mask)[0][0][0], (a + mask)[0][0][120]

(tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.]),
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         

In [56]:
class BertEncoder(nn.Module):
    def __init__(self, config):
        """BertLayerを繰り返すモジュール"""
        super().__init__()
        self.config = config
        self.layer = nn.ModuleList([BertLayer(config) for _ in range(config["num_hidden_layers"])])

    def forward(self, hidden_states, attention_mask=None, output_attention=False, output_hidden_states=False):
        """
        hidden_states: Embeddingsモジュールの出力[batch_size, seq_len, hidden]
        attention_mask: Paddingsはソフトマックスに影響を与えないようにする。[batch_size, 1, 1, seq_len]
        output_all_encoded_layers: 返り値を全TransformerBlockモジュールの出力にするか、
        それとも、最終層だけにするかのフラグ。
        output_attention: Attentionの重みを出力するか
        """

        # 返り値として使うリスト
        all_hidden_states = () if output_hidden_states else None
        all_self_attentions = () if output_attention else None

        
        # BertLayerモジュールの処理を繰り返す
        for layer_module in self.layer:
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)
                
            layer_outputs = layer_module(
                hidden_states,
                attention_mask,
                output_attention
            )
            # 返り値はいつもtupleから最初の素をとる
            hidden_states = layer_outputs[0]

            if output_attention:
                all_self_attentions = all_self_attentions + (layer_outputs[1],)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        # Noneの変数を排除する
        return tuple(
                v
                for v in [hidden_states, all_hidden_states, all_self_attentions]
                if v is not None
            )


In [57]:
a = torch.randint(3, 5, (1,256), dtype=torch.long)
embeddings = BertEmbeddings(config)

input_tensor = embeddings(a)
encoder = BertEncoder(config)

len(encoder(input_tensor)), encoder(input_tensor)[0].shape

(1, torch.Size([1, 256, 768]))

In [58]:
outputs = encoder(input_tensor, output_attention=True, output_hidden_states=True)

len(outputs), len(outputs[1]), len(outputs[2]), (outputs[0] == outputs[1][-1])

(3,
 13,
 12,
 tensor([[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]]]))

In [60]:
len(outputs[0]), outputs[2][-1].shape, outputs[1][-1].shape

(1, torch.Size([1, 12, 256, 256]), torch.Size([1, 256, 768]))

In [18]:
class BertPooler(nn.Module):
    """
    BertEncoderの出力から[CLS]の単語を取ってる、全結合層と活性関数によって
    特徴量を変換する。NextSentencePredictionや分類タスクで使う。
    """
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config["hidden_size"], config["hidden_size"])
        self.activation = nn.Tanh()

    def forward(self, hidden_states):
        # hidden_states -> [batch_size, seq_len, hidden_size]
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(first_token_tensor)
        pooled_output = self.activation(pooled_output)

        return pooled_output
        

In [19]:
pooler = BertPooler(config)

pooler(outputs[0]).shape, pooler(outputs[0])[0][0:50]

(torch.Size([1, 768]),
 tensor([ 0.0678,  0.4871, -0.7966, -0.0249, -0.1831,  0.4435,  0.4465, -0.4207,
          0.8042, -0.5632, -0.3638, -0.3969, -0.7254, -0.5836, -0.2880, -0.0489,
         -0.2480, -0.1460,  0.2313,  0.5132,  0.7189, -0.7814,  0.1698, -0.7711,
          0.4066,  0.0424,  0.0098,  0.4318,  0.9087,  0.2121, -0.5832, -0.4336,
         -0.2484,  0.1466,  0.1118,  0.5377, -0.1745,  0.0755, -0.1755, -0.0519,
          0.0561,  0.2709, -0.5778,  0.3078, -0.5558,  0.4155, -0.5731, -0.0894,
         -0.0378, -0.2992], grad_fn=<SliceBackward0>))

In [82]:
class BertModel(nn.Module):
    """Transformersのエンコーダーの部分"""
    def __init__(self, config, add_pooling_layer=True):
        super().__init__()
        self.config = config

        self.embeddings = BertEmbeddings(config)
        self.encoder = BertEncoder(config)

        self.pooler = BertPooler(config) if add_pooling_layer else None

    def forward(self, input_ids,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        output_hidden_states=False,
        output_attention=False,
        ):
        """
        input_ids: [batch_size, sequence_length]の文章の単語IDの羅列
        attention_mask: Paddingsはソフトマックスに影響を与えないようにする。[batch_size, 1, 1, seq_len]
            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.
        output_all_encoded_layers: 返り値を全TransformerBlockモジュールの出力にするか、
        それとも、最終層だけにするかのフラグ。
        output_attention: Attentionの重みを出力するか
        """
        
        # Attentionのマスクと文の1文目、2文目のidが無ければ作成する
        # このtorch.ones_likeは入力と同じタイプとディバイス
        if attention_mask is None:
            attention_mask = torch.ones_like(input_ids)
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)

        # Attention

        extended_attention_mask: torch.Tensor = get_extended_attention_mask(attention_mask)

        embedding_output = self.embeddings(input_ids=input_ids, position_ids=position_ids, token_type_ids=token_type_ids)

        # この出力は引数によって変わる。
        # output_attention=False, output_hidden_states=False -> ([batch_size, seq_len, hidden_size],)
        # output_attention=True, output_hidden_states=False -> ([batch_size, seq_len, hidden_size], 12回([batch_size, num_heads, seq_len, seq_len]))
        # output_attention=True, output_hidden_states=True -> ([batch_size, seq_len, hidden_size], 
        # 13回([batch_size, seq_len, hidden_size]), 12回([batch_size, num_heads, seq_len, seq_len])))
        encoder_outputs =  self.encoder(
            embedding_output,
            attention_mask=extended_attention_mask,
            output_attention=output_attention,
            output_hidden_states=output_hidden_states,
        )
        # これはlast_hidden_state（最後の層の出力）[batch_size, seq_len, hidden_size]
        sequence_output = encoder_outputs[0]

        # [batch_size, hidden_size]
        pooled_output = self.pooler(sequence_output) if self.pooler is not None else None

        return (sequence_output, pooled_output) + encoder_outputs[1:]


In [26]:
def get_extended_attention_mask(attention_mask: torch.Tensor, dtype: torch.float = torch.float32) -> torch.Tensor:
        """
         Attentionと因果関係のマスクをブロードキャスト可能に作成し、未来のトークンとマスクされたトークンを無視します。
        
         引数:
             attention_mask (`torch.Tensor`):
                 注目するトークンに1を示し、無視するトークンに0を示すマスク。
        
         戻り値:
             `torch.Tensor` 拡張されたAttentionマスクで、`attention_mask.dtype` と同じdtypeを持っています。

        """
        extended_attention_mask = attention_mask[:, None, None, :]

        # Attention_maskが1.0は注目させたい位置で、0.0はマスクされた位置です。
        # この操作により、注目させたい位置では0.0で、マスクされた位置ではdtypeの最小値となる
        # テンソルが作成されます。
        # これは実質的には、softmaxの前に生のスコアに追加されているため、
        # これを完全に削除するのと同じです。
        extended_attention_mask = extended_attention_mask.to(dtype=dtype) 
        extended_attention_mask = (1.0 - extended_attention_mask) * torch.finfo(dtype).min
        return extended_attention_mask


In [83]:
# 動作確認
# 入力の用意
input_ids = torch.LongTensor([[31, 51, 12, 23, 99], [15, 5, 1, 0, 0]])
attention_mask = torch.LongTensor([[1, 1, 1, 1, 1], [1, 1, 1, 0, 0]])
token_type_ids = torch.LongTensor([[0, 0, 1, 1, 1], [0, 1, 1, 1, 1]])

# Bertモデルを作成
bert = BertModel(config)

# 順伝搬させる
sequence_output, pooled_output = bert(input_ids, token_type_ids, attention_mask, output_hidden_states=False,
        output_attention=False)

print("encoded_layersのテンソルサイズ：", sequence_output.shape)
print("pooled_outputのテンソルサイズ：", pooled_output.shape)
# print("attention_probsのテンソルサイズ：", attention_probs.shape)

encoded_layersのテンソルサイズ： torch.Size([2, 5, 768])
pooled_outputのテンソルサイズ： torch.Size([2, 768])


In [84]:
outputs = bert(input_ids, token_type_ids, attention_mask, output_hidden_states=True,
        output_attention=True)

len(outputs), outputs[0].shape, outputs[1].shape, len(outputs[2]), len(outputs[3])

(4, torch.Size([2, 5, 768]), torch.Size([2, 768]), 13, 12)

In [85]:
outputs[2][-1].shape, outputs[3][-1].shape

(torch.Size([2, 5, 768]), torch.Size([2, 12, 5, 5]))

In [86]:
outputs[0] == outputs[2][-1]

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]])

In [88]:
outputs[3][-1][0]

tensor([[[0.0000, 0.0000, 0.4709, 0.3242, 0.3160],
         [0.0000, 0.0000, 0.4529, 0.3695, 0.0000],
         [0.0000, 0.0000, 0.3338, 0.4519, 0.0000],
         [0.0000, 0.0000, 0.3516, 0.4158, 0.3438],
         [0.0000, 0.0000, 0.4611, 0.3391, 0.3110]],

        [[0.0000, 0.0000, 0.3763, 0.3624, 0.3724],
         [0.0000, 0.0000, 0.5063, 0.0000, 0.2820],
         [0.0000, 0.0000, 0.3646, 0.3918, 0.3547],
         [0.0000, 0.0000, 0.0000, 0.4057, 0.3197],
         [0.0000, 0.0000, 0.3702, 0.4167, 0.3243]],

        [[0.0000, 0.0000, 0.0000, 0.3620, 0.3143],
         [0.0000, 0.0000, 0.3126, 0.5222, 0.2763],
         [0.0000, 0.0000, 0.3017, 0.4340, 0.3754],
         [0.0000, 0.0000, 0.4176, 0.3767, 0.3169],
         [0.0000, 0.0000, 0.3645, 0.3814, 0.3652]],

        [[0.0000, 0.0000, 0.3520, 0.3885, 0.3706],
         [0.0000, 0.0000, 0.3105, 0.3840, 0.4166],
         [0.0000, 0.0000, 0.3473, 0.3610, 0.4029],
         [0.0000, 0.0000, 0.3274, 0.3984, 0.3853],
         [0.0000, 0.0000,